<a href="https://colab.research.google.com/github/flaviusfetean/method_name_predictor/blob/main/nlp_codet5_method_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Library installation

In [1]:
!pip install accelerate peft bitsandbytes transformers trl

In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [3]:
!pip install pytorch-lightning

#Import and check the dataset

The dataset is a collection of methods extracted from the java files of the https://github.com/JetBrains/intellij-community repo. It is stored as a json file with dictionaries containing 3 details about a method: **body, name and class** (the last one was extracted because at a deeper look it seemed that for a lot of methods, their name was uncorrelated to the body (only `return true`, or smth like that), thus the class name could give a clue about `"what is true?"`, considering that in a real-life scenario, the class name info is available for method name inferrence. After experimentation, training with class info yielded poorer results, so in the current training it is not present)

  Methods from the repo that were for testing purposes (i.e.: had "test" inside their name) were not added to the dataset, as their bodies were often completely unrelated to their names (ex: had 1/2/3 in the names only bcause of the ordering - more code context would have been necessary for such a prediction)

In [4]:
from datasets import load_dataset
import os
methods_dataset = load_dataset("json", data_files={"train":"json_clean_train.json", "test": "json_clean_test.json"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
print(methods_dataset)

DatasetDict({
    train: Dataset({
        features: ['name', 'body', 'class'],
        num_rows: 99991
    })
    test: Dataset({
        features: ['name', 'body', 'class'],
        num_rows: 33331
    })
})


#Preprocess the dataset

Utility method for converting the dataset given as dictionaries of strings into tokens which can be fed to the model

In [6]:
from transformers import RobertaTokenizer

#load the tokenizer specific to the reference model - codet5-small
tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-small")

max_input_length = 1024
max_target_length = 128

def convert_dataset(raw_dataset):
    # encode the body-name pairs
    classes = raw_dataset['class']
    bodies = raw_dataset['body']
    names = raw_dataset['name']

    model_inputs = tokenizer(bodies, max_length=max_input_length, padding="max_length", truncation=True)

    # encode the method names
    labels = tokenizer(names, max_length=max_target_length, padding="max_length", truncation=True).input_ids

    # important: we need to replace the index of the padding tokens by -100
    # such that they are not taken into account by the CrossEntropyLoss
    labels_with_ignore_index = []
    for labels_example in labels:
        labels_example = [label if label != 0 else -100 for label in labels_example]
        labels_with_ignore_index.append(labels_example)

    model_inputs["labels"] = labels_with_ignore_index

    return model_inputs

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

In [7]:
dataset = methods_dataset.map(convert_dataset, batched=True)

Map:   0%|          | 0/99991 [00:00<?, ? examples/s]

Map:   0%|          | 0/33331 [00:00<?, ? examples/s]

In [8]:
#check if the preprocessing added the necessary columns
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['name', 'body', 'class', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 99991
    })
    test: Dataset({
        features: ['name', 'body', 'class', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 33331
    })
})


In [9]:
#load the dataset in the dataloaders
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8)
#test_dataloader = DataLoader(dataset['test'], batch_size=2)

Check if random bodies in the dataset have the correct format, which would be the code followed by `<pad>` until 1024 tokens are reached

In [10]:
#example input
batch = next(iter(train_dataloader))
print(batch.keys())
tokenizer.decode(batch['input_ids'][0])

dict_keys(['input_ids', 'attention_mask', 'labels'])


'<s>return false;</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [11]:
#example label
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

'<s>hasModifierProperty</s>'

#Load and train the transformer

In [12]:
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl
import torch

class CodeT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=10, warmup_steps=1000):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small", device_map={"": 0})
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs

    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss

    def training_step(self, batch, batch_idx):
        torch.set_grad_enabled(True)
        loss = self.common_step(batch, batch_idx)
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                                     num_warmup_steps=self.hparams.warmup_steps,
                                                                     num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}

        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader


In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

model = CodeT5()

lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = Trainer(callbacks=[lr_monitor], max_epochs=5)
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
----------

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
model.model.save_pretrained('/content/drive/MyDrive/llama_output/codet5_method_predictor_5epochs')

#Manually test the resulting model

In [13]:
#load the checkpointed model
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/llama_output/codet5_method_predictor",
                                                    device_map={"": 0})
#along with the original model to be able to compare
original_model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small",  cache_dir=r"/content/drive/MyDrive/llama_output",
                                                                device_map={"": 0})

Sample some random inputs from the test set (by entering a random index in the first line of the next cell) and feed them to both networks to compare the outputs. In some cases, the fine-tuned model gets the answer exactly right, in most cases it gets some words right, and in some it does not get anything right. The original model, however, even if prompted with `Generate a name for the body: {}`, rarely gets it right

In [14]:
# prepare input for the model
test_dict = methods_dataset['test'][22]
print(f"Inferencing for body: {test_dict['body']}")
input_ids = tokenizer(test_dict['body'], return_tensors='pt').input_ids.to('cuda')
input_ids_orig = tokenizer("Generate a method name for the body: " + test_dict['body'], return_tensors='pt').input_ids.to('cuda')
# generate
outputs = model.generate(input_ids)
outputs_original = original_model.generate(input_ids_orig)
print("Generated name by the fine-tuned model:", tokenizer.decode(outputs[0], skip_special_tokens=True))
print("Generated name by the original model:", tokenizer.decode(outputs_original[0], skip_special_tokens=True))
print("Ground truth: ", test_dict['name'])

Inferencing for: setErrorMessage(errorMessage);


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated name by the fine-tuned model: setErrorMessage
Generated name by the original model:  public void
Ground truth:  errorOccurred


#Testing the result model


1.   **Hard Comparison**:
  The output is tested for an exact match

In [15]:
def compare_outputs(pred, gt):
    return 1 if pred == gt else 0



2.   **Soft comparison (similarity)**: We will count the number of words that appear in both the output and the ground truth, as the output may still have relevance even if not an exact match

In [16]:
def split_camel_case(input_string):
    """
    Method to split a method name which is known to be a camel-case
    into its composing words (Java convention)
    """
    try:
        words = [input_string[0]]

        for char in input_string[1:]:
            if char.isupper():
                words.append(char.lower())
            else:
                words[-1] += char
    except IndexError:
        return ""

    return ' '.join(words)

camel_case_string = "camelCaseExample"
result = split_camel_case(camel_case_string)
print(result)

camel case example


In [17]:

def compare_similarity(pred, gt):
    """Often the method name is not predicted exactly the same as the ground truth
    But it is composed of some words that are also present in the ground truth
    Therefore, we will consider the similarity between the results as the number of words in the ground truth that are also present in the prediction divided by the maximum length of the two strings
    """

    max_similarity = 0
    words = split_camel_case(gt).split()
    for word in words:
        if word in pred.lower():
            max_similarity += len(word)

    return max_similarity / max(len(pred), len(gt))



3.   **ROUGE score**: A generalization of the soft score, will also take into account bigrams and longest-common-sequences. Is mainly used for translation and summarization tasks to evaluate the similarity of 2 or more texts, and is helpful for this use-case, as a method name can be viewed as a very short summary of a code



In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install evaluate absl-py rouge_score nltk

In [19]:
import evaluate

rouge = evaluate.load('rouge')

def compare_rouge(preds, gts):
    """
    Rouge will treat the texts as summaries, so we will have to split
    the method names into composing words and treat them as summaries
    """
    pred_split = [split_camel_case(pred) for pred in preds]
    gt_split = [split_camel_case(gt) for gt in gts]

    return rouge.compute(predictions=pred_split, references=gt_split)

print(compare_rouge(["getTestDefault"], ["myTestNotDefault"])) #Expected 0.57 r1, 0 r2, 0.57 rl

{'rouge1': 0.5714285714285715, 'rouge2': 0.0, 'rougeL': 0.5714285714285715, 'rougeLsum': 0.5714285714285715}


In [20]:
#test the compare functions
print(compare_outputs("hello", "hello")) #Expected 1
print(compare_outputs("hello", "world")) #Expected 0
print(compare_similarity("hello", "hello")) #Expected 1.0
print(compare_similarity("hello", "hell")) #Expected 0.8
print(compare_similarity("hello", "helll")) #Expected 0.0
print(compare_similarity("getTestDefault", "myTestNotDefault")) #Expected 0.687

1
0
1.0
0.8
0.0
0.6875


Method to predict on all inputs, given a model and a dataset (dataset must be formatted for the model input)

In [21]:
from tqdm import tqdm

def predict_all(model, dataset, tokenizer, max_new_tokens=20):
    #turn of learning functions
    model.eval()
    torch.no_grad()

    #innitialize lists for predictions and ground truths that will be comppared
    predictions = []
    gt = []
    for i, example in enumerate(dataset):
        #prepare input
        input_ids = tokenizer(example['input'], return_tensors='pt').input_ids.to('cuda')
        # generate
        outputs = model.generate(input_ids, max_new_tokens=max_new_tokens)
        #decode predicate and add to list
        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(pred)
        gt.append(example['label'])
        if i%1000 == 0:
            print(f"{i}/{len(dataset)}")
    return predictions, gt

Cell to evaluate predictions on soft, hard and rouge scores

In [22]:
#evaluate the predictions

def evaluate_predictions(predictions, gt):
    hard_score = 0
    soft_score = 0

    for i, (pred, gndt) in enumerate(zip(predictions, gt)):
        hard_score += compare_outputs(pred, gndt)
        soft_score += compare_similarity(pred, gndt)

    print("Hard score: ", hard_score / len(predictions))
    print("Soft score: ", soft_score / len(predictions))
    print("Rouge score: ", compare_rouge(predictions, gt))

Evaluate the fine-tuned model

In [ ]:
#get all predictions
model_input = [{"input": example['body'], "label": example['name']} for example in methods_dataset['test']]
predictions, gt = predict_all(model, model_input, tokenizer)

In [24]:
evaluate_predictions(predictions, gt)

Hard score:  0.3815667096696769
Soft score:  0.5165091046805851
Rouge score:  {'rouge1': 0.5483053927827923, 'rouge2': 0.34518662275340956, 'rougeL': 0.5475875869594452, 'rougeLsum': 0.5478755418632586}


Evaluate the original model

In [ ]:
model_input = [{"input": "Generate a name for the method having the following body: "+example['body'], "label": example['name']} for example in methods_dataset['test']]
predictions_orig, gt = predict_all(original_model, model_input, tokenizer)

In [26]:
evaluate_predictions(predictions_orig, gt)

Hard score:  0.0014701029072035042
Soft score:  0.11355751830934176
Rouge score:  {'rouge1': 0.12056786261095578, 'rouge2': 0.04686184785875327, 'rougeL': 0.11879224584656178, 'rougeLsum': 0.1188859370058751}
